In [1]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import font

In [2]:
from movie import Movie

In [3]:
from parse import parse_imdb_data

In [4]:
movies = parse_imdb_data()

PARSING THE RATINGS FILE
PARSING THE GENRES FILE
PARSING THE KEYWORDS FILE


In [5]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>10000, m_))
m_=list(filter(lambda m: m.mean_rating>7.5, m_))
len(m_)

2169

In [6]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(data[0][0].isnumeric()): # FIX: doesn't work for float
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

In [7]:
min_year = min_rating_count = min_mean_rating = float('inf')
max_year = max_rating_count = max_mean_rating = float('-inf')
for m in m_:
    min_year = min((min_year, m.year))
    max_year = max((max_year, m.year))
    min_rating_count = min((min_rating_count, m.num_ratings))
    max_rating_count = max((max_rating_count, m.num_ratings))
    min_mean_rating = min((min_mean_rating, m.mean_rating))
    max_mean_rating = max((max_mean_rating, m.mean_rating))    

In [56]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky=(N, E))
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'

yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky=(N, E))
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=IntVar()
minyear.set(min_year)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=IntVar()
maxyear.set(max_year)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky=(N, E))
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
minratingcount.set(min_rating_count)
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(max_rating_count)
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
meanratingfilterframe.grid(column=0, row=2, sticky=(N, E))
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(min_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(max_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

def filter_data(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))    
    for iid in data:
        # Year
        if float(data_tree.set(iid)['Year']) < minyear.get():
            remove_iids.add(iid)
        if float(data_tree.set(iid)['Year']) > maxyear.get():
            remove_iids.add(iid)
        # Rating Count
        if float(data_tree.set(iid)['Rating Count']) < minratingcount.get():
            remove_iids.add(iid)
        if float(data_tree.set(iid)['Rating Count']) > maxratingcount.get():
            remove_iids.add(iid)
        # MEAN RATING
        if float(data_tree.set(iid)['Mean Rating']) < minrating.get():
            remove_iids.add(iid)
        if float(data_tree.set(iid)['Mean Rating']) > maxrating.get():
            remove_iids.add(iid)
    #data_tree.delete(list(remove_iids))
    for iid in remove_iids:
        data_tree.delete(iid)
    #TODO
    # 1. Update the other fields based on the new filtered data
    # 2. There is no way to go back in filtering right now

def reset_filters(*args):
    print(args)
    
def search_filter(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))    
    for iid in data:
        # Year
        if data_tree.set(iid)['Title'].find(searchphrase.get()) == -1:
            remove_iids.add(iid)        
    for iid in remove_iids:
        data_tree.delete(iid)
    
def double_click(e):
    item = data_tree.selection()[0]
    #print("You clicked on", data_tree.set(item))
    #print(m_[int(item)])
    tl = Toplevel(root)
    #ttk.Label(tl, text=data_tree.set(item)).grid(column=0, row=0, sticky="E")
    m = m_[int(item)]
    
    ttk.Label(tl, text=m.title + " (" + str(m.year) + ")", font=font.Font(family='Helvetica', size=14, weight='bold')).grid(column=0, row=0, sticky="W")
    if m.genres is not None:
        g = ", ".join(m.genres)
        ttk.Label(tl, text="Genres: " + g).grid(column=0, row=1, sticky="W")      
    if m.keywords is not None:
        #cnames = StringVar(value=countrynames)
        kl= Listbox(tl, listvariable=StringVar(value=m.keywords), height=5)
        kl.grid(column=0, row=2, sticky="nsew")
        vs = ttk.Scrollbar(tl, orient=VERTICAL, command=kl.yview)
        kl.configure(yscrollcommand=vs.set)
        vs.grid(column=1, row=2, sticky="nsew")
        hs = ttk.Scrollbar(tl, orient=HORIZONTAL, command=kl.xview)
        kl.configure(xscrollcommand=hs.set)
        hs.grid(column=0, row=3, sticky="nsew")
    

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=3, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=1900, to=2017 , showvalue=True, label="Minimum Year", command=filter_year)
#s.grid(column=0, row=0, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0, to=1e3, resolution=1, showvalue=True, label="Minimum # Ratings", command=filter_num_ratings)
#s.grid(column=0, row=1, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0.0, to=10.0, resolution=0.1, showvalue=True, label="Minimum Mean Rating")
#s.grid(column=0, row=2, sticky=W)




dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky="nsew")
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

searchfilterframe = ttk.Frame(dataframe, padding="3 3 12 12")
searchfilterframe['borderwidth'] = 2
searchfilterframe['relief'] = 'sunken'
searchfilterframe.grid(column=0, row=0, sticky="nsew")

searchphrase = StringVar()
ttk.Entry(searchfilterframe, textvariable=searchphrase).grid(column=0, row=0, sticky="nsew")
ttk.Button(searchfilterframe, text="Search", command=search_filter).grid(column=1, row=0, sticky="nsew")

# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating"]
#data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings", selectmode='browse')

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
data_tree.grid(column=0, row=1, sticky='nsew')

data_tree.bind("<Double-1>", double_click)

vsb.grid(column=1, row=1, sticky='ns')
hsb.grid(column=0, row=2, sticky='ew')

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_rowconfigure(0, weight=1)


for col in data_header:
    data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for i in range(len(m_)):
    item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating]
    data_tree.insert('', 'end', iid = str(i), values=item)
    # adjust column's width if necessary to fit each value
    for ix, val in enumerate(item):
        col_w = tkFont.Font().measure(val)
        if data_tree.column(data_header[ix],width=None)<col_w:
            data_tree.column(data_header[ix], width=col_w)

root.mainloop()